In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np

from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
from waffles.np02_utils.PlotUtils import runBasicWfAnaNP02, plot_detectors, fithist
from waffles.np02_utils.load_utils import open_processed

In [ ]:
run = 42819
nwaveforms = 80000
dettype = "pmt"

## Only change if necessary
output_dir = f"/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/commissioning/"
ch = {}
det = "VD_PMT_PDS" 
endpoint = 110
trigger = "self_trigger"

In [ ]:
wfset_full = open_processed(run, dettype, output_dir, nwaveforms=nwaveforms)
wfset_full

In [ ]:
from collections import Counter
timestamps = sorted([ wf.timestamp for wf in wfset_full.waveforms ])
c = Counter(timestamps)
print(f"Total number of timestamps: {len(c)}")
matchtimestamps = [ k for k in c if c[k] >= len(list(wfset_full.available_channels[run][endpoint])) ]
# matchtimestamps = [ k for k in c if c[k] >= 20 ]
print(f"Remaining timestamps: {len(matchtimestamps)}")

In [ ]:
def get_external(waveform: Waveform, validtimes = []) -> bool:
    if np.any((waveform.adcs[0:500] > 16000) | (waveform.adcs[0:500] < 100) ):
        return False

    if waveform.timestamp not in validtimes:
        return False
    
    return True

matchingtimes = []
print(wfset_full)
wfset_full = WaveformSet.from_filtered_WaveformSet(wfset_full, get_external, matchtimestamps, show_progress=True)
print(wfset_full)

In [ ]:
runBasicWfAnaNP02(wfset_full, baselinefinish=240, onlyoptimal=True, threshold=25)

In [ ]:
argsheat = dict(
    mode="heatmap",
    analysis_label="std",
    adc_range_above_baseline=1000,
    adc_range_below_baseline=-200,
    adc_bins=200,
    time_bins=wfset_full.points_per_wf//2,
    filtering=4,
    share_y_scale=False,
    share_x_scale=True,
    wfs_per_axes=500,
    zlog=True,
    width=1600,
)
pmtuch = list(wfset_full.available_channels[run][110])
detector=[ UniqueChannel(110, ch) for ch in pmtuch ]
# plot_detectors(wfset_full, detector, plot_function=fithist, html=None, wf_func=histargs)
# plot_detectors(wfset_full, detector, **argsheat)
          
detector=[ UniqueChannel(110, ch) for ch in pmtuch[:len(pmtuch)//2] ]
plot_detectors(wfset_full, detector, **argsheat)
detector=[ UniqueChannel(110, ch) for ch in pmtuch[len(pmtuch)//2:] ]
plot_detectors(wfset_full, detector, **argsheat)

In [ ]:
histargs = dict(    
    doprocess = True,
    dofit = True,
    variable = 'integral',
    show_progress = False,
    configyaml="ch_snr_parameters_pmts.yaml",
    fitmultigauss=True,
    # log_y=True,
    normalize_hist=True,
    # verbosemultigauss = True,
)
argsplot = dict(
    width=1600,
)
pmtuch = list(wfset_full.available_channels[run][endpoint])
# detector=[ UniqueChannel(110, ch) for ch in pmtuch ]
# detector=[ UniqueChannel(endpoint, 12) ]

detector=[ UniqueChannel(endpoint, ch) for ch in pmtuch[:len(pmtuch)//2] ]
v = plot_detectors(wfset_full, detector, plot_function=fithist, wf_func=histargs, **argsplot)

#detector=[UniqueChannel(110,44),UniqueChannel(110,46)]
detector=[ UniqueChannel(endpoint, ch) for ch in pmtuch[len(pmtuch)//2:] ]
plot_detectors(wfset_full, detector, plot_function=fithist, wf_func=histargs, **argsplot)

In [ ]:
wfch = ChannelWsGrid.clusterize_waveform_set(wfset_full)
for ch in sorted(pmtuch):
    wfs = wfch[endpoint][ch]
    try:
        normalization = wfs.waveforms[0].analyses['std'].result['normalization']
        gain = normalization*wfs.waveforms[0].analyses['std'].result['gain']
        errgain = normalization*wfs.waveforms[0].analyses['std'].result['errgain']
        snr = wfs.waveforms[0].analyses['std'].result['snr']
    except:
        snr = np.nan
        gain = np.nan
        errgain = np.nan
    print(f"{ch}, {snr:.2f}, {gain:.2f}, {errgain:.2f}")